In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

## 3D meshing example

This notebook shows how to

1. Load and visualize a volume
1. Apply image filters to and segment image
1. Generate a 3D surface mesh from the binary image
1. Decimate/simplify a 3D mesh
1. Visualize and save the mesh to gmsh22 format

### Todo:

- Generate 3D volume mesh
- Surface mesh must be 'watertight'
- Constrained Delaunay triangulation to fix surface (do not go outside surface), i.e.
https://notebook.community/daniel-koehn/Theory-of-seismic-waves-II/02_Mesh_generation/4_Tri_mesh_delaunay_yigma_tepe,
https://wias-berlin.de/software/tetgen/1.5/doc/manual/manual002.html#sec7

In [2]:
from nanomesh.volume import Volume
import pyvista as pv
from skimage import filters

In [3]:
vol = Volume.load('sample_data.npy')
vol_gauss = vol.apply(filters.gaussian, sigma=5)
thresh = filters.threshold_li(vol_gauss.image)

seg_vol = Volume(1.0 * (vol_gauss.image >= thresh))
seg_vol.show_slice()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='index', max=199), Dropdown(description='along', options=…

### Generate 3d tetragonal mesh

In [4]:
import numpy as np

def simplify_mesh_trimesh(vertices, faces, n_faces):
    """Simplify mesh using trimesh."""
    import trimesh
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    decimated = mesh.simplify_quadratic_decimation(n_faces)
    return decimated

def simplify_mesh_open3d(vertices: np.ndarray, faces: np.ndarray, n_faces):
    """Simplify mesh using open3d."""
    import open3d
    o3d_verts = open3d.utility.Vector3dVector(vertices)
    o3d_faces = open3d.utility.Vector3iVector(faces)
    o3d_mesh = open3d.geometry.TriangleMesh(o3d_verts, o3d_faces)

    o3d_new_mesh = o3d_mesh.simplify_quadric_decimation(n_faces)
    
    new_verts = np.array(o3d_new_mesh.vertices)
    new_faces = np.array(o3d_new_mesh.triangles)
    
    decimated = meshio.Mesh(points=new_verts, cells=[('triangle', new_faces)])
    return decimated

In [88]:
import pyvista as pv

def meshio_to_polydata(mesh):
    return pv.from_meshio(mesh)

def meshio_to_trimesh():
    raise NotImplementedError

def trimesh_to_meshio():
    raise NotImplementedError

def trimesh_to_polydata():
    raise NotImplementedError

def polydata_to_meshio():
    raise NotImplementedError

def polydata_to_trimesh():
    raise NotImplementedError

def tetrahedra_to_mesh(points, faces, mask):
    """Convert list of tetrahedra and mask to meshio.Mesh"""
    if mask is not None:
        faces = faces[mask]

    cells = [
        ('tetra', faces),
    ]

    mesh = meshio.Mesh(points, cells)
    mesh.remove_orphaned_nodes()
    
    return mesh

def triangles_to_mesh(points, faces, mask=None):
    """Convert list of triangles and mask to meshio.Mesh"""
    if mask is not None:
        faces = faces[mask]
    
    cells = [
        ('triangle', faces),
    ]

    mesh = meshio.Mesh(points, cells)
    mesh.remove_orphaned_nodes()
    
    return mesh

In [126]:
def remove_pad_from_mesh(points, tetrahedra, pad_width, shape):
    centers = points[tetrahedra].mean(1)
    dimx, dimy, dimz = np.array(shape) - pad_width*2

    maskx = (centers[:, 2] >= pad_width) & (centers[:, 2] <= dimx - pad_width)
    masky = (centers[:, 1] >= pad_width) & (centers[:, 1] <= dimy - pad_width)
    maskz = (centers[:, 0] >= pad_width) & (centers[:, 0] <= dimz - pad_width)

    mask = maskx * masky * maskz
    mask = mask.nonzero()[0]
    
    return_mesh = tetrahedra_to_mesh(points, tetrahedra, mask)
    return_mesh.remove_orphaned_nodes

    return return_mesh

In [195]:
from skimage import measure
from scipy.spatial import Delaunay
import meshio
from nanomesh.mesh3d import add_points_kmeans_sklearn

image = seg_vol.image

point_density = 1/10000
scale = 0.5
plot = False
pad_width = 5

points = []

if point_density:
    # grid_points = add_points_grid(image, border=5)
    n_points1 = int(np.sum(image == 1) * point_density)
    grid_points = add_points_kmeans_sklearn(
        image, 
        iters=10, 
        n_points=n_points1, 
        scale=scale)
    points.append(grid_points)

    # adding points to holes helps to get a cleaner result
    n_points0 = int(np.sum(image == 0) * point_density)
    grid_points = add_points_kmeans_sklearn(
        1 - image,
        iters=10,
        n_points=n_points0,
        scale=scale)
    points.append(grid_points)

if pad_width:
    image = np.pad(image, pad_width, mode='edge')

verts, faces, normals, values = measure.marching_cubes(
    image, 
    allow_degenerate=False,
    step_size=5,
)

mesh = simplify_mesh_trimesh(vertices=verts, faces=faces, n_faces=5000)

points.append(mesh.vertices)
points = np.vstack(points)

tetrahedra = Delaunay(points, incremental=False).simplices

centers = points[tetrahedra].mean(1)

pore_mask = image[tuple(centers.astype(int).T)] == 1

masks = [pore_mask]

if pad_width:
    dimx, dimy, dimz = np.array(image.shape) - pad_width*2

    maskx = (centers[:, 2] >= pad_width) & (centers[:, 2] <= dimx - pad_width)
    masky = (centers[:, 1] >= pad_width) & (centers[:, 1] <= dimy - pad_width)
    maskz = (centers[:, 0] >= pad_width) & (centers[:, 0] <= dimz - pad_width)
    
    masks.extend([maskx, masky, maskz])
    
mask = np.product(masks, axis=0).astype(bool)

mesh = tetrahedra_to_mesh(points, tetrahedra, mask)
mesh.remove_orphaned_nodes()

pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [194]:
mesh = tetrahedra_to_mesh(points, tetrahedra, np.product(masks, axis=0).astype(bool))
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [196]:
def show_submesh(*grids, index=100, along='x', plotter=pv.PlotterITK):
    """Slow a slice of the mesh."""
    plotter = plotter()

    for grid in grids:
        # get cell centroids
        cells = grid.cells.reshape(-1, 5)[:, 1:]
        cell_center = grid.points[cells].mean(1)

        # extract cells below index
        axis = 'zyx'.index(along)

        mask = cell_center[:, axis] < index
        cell_ind = mask.nonzero()[0]
        subgrid = grid.extract_cells(cell_ind)

        plotter.add_mesh(subgrid)
    
    plotter.show()
        
    return plotter

grid = meshio_to_polydata(mesh)
show_submesh(grid, index=100)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…